In [8]:
import pandas as pd

# Creating a DataFrame from the dataset
data = {
    "Buying": ["med", "med", "med", "med", "med", "low", "low", "low", "low", "low", 
               "low", "low", "low", "low", "low", "low", "low", "low", "low", "low", 
               "low", "low", "low", "low", "low", "low", "low", "low", "low", "low"],
    "Price": ["med", "med", "med", "med", "med", "low", "low", "low", "low", "low", 
              "low", "low", "low", "low", "low", "low", "low", "low", "low", "low", 
              "low", "low", "low", "low", "low", "low", "low", "low", "low", "low"],
    "Maintenance": ["med", "med", "med", "med", "med", "low", "low", "low", "low", "low", 
                    "low", "low", "low", "low", "low", "low", "low", "low", "low", "low", 
                    "low", "low", "low", "low", "low", "low", "low", "low", "low", "low"],
    "Doors": [2, 2, 3, 3, 3, "5more", "5more", "5more", "5more", "5more", "5more", 
              "5more", "5more", "4", "4", "4", "4", "4", "4", "4", "5more", "5more", 
              "5more", "5more", 2, 2, 3, 3, 3, 4],
    "Persons": [4, "more", 4, "more", "more", 2, 2, 4, 4, 4, 4, "more", "more", 4, 4, 
                "more", "more", "4", 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, "more"],
    "Lug_Boot": ["big", "big", "big", "big", "big", "big", "big", "small", "med", "big", 
                 "small", "med", "small", "med", "med", "small", "med", "big", "small", 
                 "big", "small", "small", "small", "small", "med", "small", "med", "small", 
                 "big", "small"],
    "Safety": ["vgood", "vgood", "vgood", "vgood", "vgood", "unacc", "unacc", "unacc", 
               "unacc", "unacc", "unacc", "unacc", "unacc", "good", "good", "good", "good", 
               "good", "good", "good", "good", "good", "good", "good", "good", "acc", "acc", 
               "acc", "acc", "acc"]
}

df = pd.DataFrame(data)

# Target class
target = "Safety"


In [4]:
pip install pandas numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 2.8 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
import numpy as np
import pandas as pd

def calculate_entropy(df, target):
    """Calculate entropy of the target class."""
    class_counts = df[target].value_counts()
    probabilities = class_counts / len(df)
    entropy = -np.sum(probabilities * np.log2(probabilities))
    return entropy

def split_data(df, attribute):
    """Split dataset by the given attribute."""
    return df.groupby(attribute)

def best_split(df, attributes, target):
    """Choose the best attribute to split on."""
    base_entropy = calculate_entropy(df, target)
    best_info_gain = -1
    best_attribute = None
    
    for attribute in attributes:
        subsets = split_data(df, attribute)
        weighted_entropy = 0
        
        for _, subset in subsets:
            weighted_entropy += (len(subset) / len(df)) * calculate_entropy(subset, target)
        
        info_gain = base_entropy - weighted_entropy
        if info_gain > best_info_gain:
            best_info_gain = info_gain
            best_attribute = attribute
    
    return best_attribute

def tdidt(df, attributes, target, max_depth=5, depth=0):
    """Build a decision tree using TDIDT."""
    # If all instances are of the same class, return that class
    if df[target].nunique() == 1:
        return df[target].iloc[0]
    
    # If max depth is reached, return the most frequent class
    if depth == max_depth:
        return df[target].mode()[0]
    
    # Choose the best attribute to split on
    best_attribute = best_split(df, attributes, target)
    
    # Split the dataset based on the best attribute
    tree = {best_attribute: {}}
    subsets = split_data(df, best_attribute)
    
    for value, subset in subsets:
        tree[best_attribute][value] = tdidt(subset, [attr for attr in attributes if attr != best_attribute], target, max_depth, depth + 1)
    
    return tree


In [12]:
# Define the list of attributes and build the decision tree using TDIDT
attributes = ["Buying", "Price", "Maintenance", "Doors", "Persons", "Lug_Boot"]
tree_take_first = tdidt(df, attributes, target)

print("Decision Tree (Take First):")
print(tree_take_first)


Decision Tree (Take First):
{'Doors': {2: {'Lug_Boot': {'big': 'vgood', 'med': 'good', 'small': 'acc'}}, 3: {'Buying': {'low': 'acc', 'med': 'vgood'}}, 4: 'acc', '4': 'good', '5more': {'Lug_Boot': {'big': 'unacc', 'med': 'unacc', 'small': {'Persons': {4: {'Buying': {'low': {'Price': {'low': 'good'}}}}, 'more': 'unacc'}}}}}}


In [14]:
# Reversing the attributes to simulate "Take Last" approach
attributes_last = list(reversed(attributes))
tree_take_last = tdidt(df, attributes_last, target)

print("\nDecision Tree (Take Last):")
print(tree_take_last)



Decision Tree (Take Last):
{'Doors': {2: {'Lug_Boot': {'big': 'vgood', 'med': 'good', 'small': 'acc'}}, 3: {'Maintenance': {'low': 'acc', 'med': 'vgood'}}, 4: 'acc', '4': 'good', '5more': {'Lug_Boot': {'big': 'unacc', 'med': 'unacc', 'small': {'Persons': {4: {'Maintenance': {'low': {'Price': {'low': 'good'}}}}, 'more': 'unacc'}}}}}}


In [18]:
def extract_rules(tree, parent_rule=None):
    if parent_rule is None:
        parent_rule = []  # Initialize as an empty list when it's the first call
    
    rules = []
    
    # Check if the current node is a leaf node (dictionary with only one key, which is the class label)
    if isinstance(tree, str):  # Leaf node, return class label
        rules.append(" AND ".join(parent_rule) + f" => {tree}")
    else:
        # If it's not a leaf, it’s a decision node (has attributes to split on)
        attribute = list(tree.keys())[0]
        for value, subtree in tree[attribute].items():
            rule = parent_rule + [f"{attribute} = {value}"]
            rules += extract_rules(subtree, rule)  # Recurse for the subtree
    return rules


In [20]:
import numpy as np
import pandas as pd

# Dataset
data = {
    "Buying": ["med", "med", "med", "med", "med", "low", "low", "low", "low", "low", 
               "low", "low", "low", "low", "low", "low", "low", "low", "low", "low", 


SyntaxError: incomplete input (2779101741.py, line 7)

In [22]:
import pandas as pd
import numpy as np
from collections import Counter

# Dataset
data = {
    "Buying": ["med", "med", "med", "med", "med", "low", "low", "low", "low", "low", 
               "low", "low", "low", "low", "low", "low", "low", "low", "low", "low", 
               "low", "low", "low", "low", "low", "low", "low", "low", "low", "low"],
    "Price": ["med", "med", "med", "med", "med", "low", "low", "low", "low", "low", 
              "low", "low", "low", "low", "low", "low", "low", "low", "low", "low", 
              "low", "low", "low", "low", "low", "low", "low", "low", "low", "low"],
    "Maintenance": ["med", "med", "med", "med", "med", "low", "low", "low", "low", "low", 
                    "low", "low", "low", "low", "low", "low", "low", "low", "low", "low", 
                    "low", "low", "low", "low", "low", "low", "low", "low", "low", "low"],
    "Doors": [2, 2, 3, 3, 3, "5more", "5more", "5more", "5more", "5more", "5more", 
              "5more", "5more", "4", "4", "4", "4", "4", "4", "4", "5more", "5more", 
              "5more", "5more", 2, 2, 3, 3, 3, 4],
    "Persons": [4, "more", 4, "more", "more", 2, 2, 4, 4, 4, 4, "more", "more", 4, 4, 
                "more", "more", "4", 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, "more"],
    "Lug_Boot": ["big", "big", "big", "big", "big", "big", "big", "small", "med", "big", 
                 "small", "med", "small", "med", "med", "small", "med", "big", "small", 
                 "big", "small", "small", "small", "small", "med", "small", "med", "small", 
                 "big", "small"],
    "Safety": ["vgood", "vgood", "vgood", "vgood", "vgood", "unacc", "unacc", "unacc", 
               "unacc", "unacc", "unacc", "unacc", "unacc", "good", "good", "good", "good", 
               "good", "good", "good", "good", "good", "good", "good", "good", "acc", "acc", 
               "acc", "acc", "acc"]
}

df = pd.DataFrame(data)

# Target class
target = "Safety"

# TDIDT Algorithm

def entropy(data):
    """Calculate the entropy of a dataset."""
    class_counts = Counter(data)
    total = len(data)
    return -sum((count/total) * np.log2(count/total) for count in class_counts.values())

def information_gain(data, attribute, target):
    """Calculate the information gain of a dataset for a given attribute."""
    total_entropy = entropy(data[target])
    values = data[attribute].unique()
    
    weighted_entropy = 0
    for value in values:
        subset = data[data[attribute] == value]
        weighted_entropy += (len(subset) / len(data)) * entropy(subset)
    
    return total_entropy - weighted_entropy

def tdidt(data, attributes, target):
    """Implement the TDIDT algorithm for decision tree learning."""
    # If all data belongs to one class, return that class
    if len(set(data[target])) == 1:
        return data[target].iloc[0]
    
    # If no attributes are left to split on, return the most common class
    if not attributes:
        return data[target].mode()[0]
    
    # Choose the best attribute based on information gain
    gains = [(attribute, information_gain(data, attribute, target)) for attribute in attributes]
    best_attribute, _ = max(gains, key=lambda x: x[1])
    
    tree = {best_attribute: {}}
    remaining_attributes = [attr for attr in attributes if attr != best_attribute]
    
    for value in data[best_attribute].unique():
        subset = data[data[best_attribute] == value]
        tree[best_attribute][value] = tdidt(subset, remaining_attributes, target)
    
    return tree

# Extracting rules from the decision tree
def extract_rules(tree, parent_rule=None):
    if parent_rule is None:
        parent_rule = []  # Initialize as an empty list when it's the first call
    
    rules = []
    
    # Check if the current node is a leaf node (dictionary with only one key, which is the class label)
    if isinstance(tree, str):  # Leaf node, return class label
        rules.append(" AND ".join(parent_rule) + f" => {tree}")
    else:
        # If it's not a leaf, it’s a decision node (has attributes to split on)
        attribute = list(tree.keys())[0]
        for value, subtree in tree[attribute].items():
            rule = parent_rule + [f"{attribute} = {value}"]
            rules += extract_rules(subtree, rule)  # Recurse for the subtree
    return rules

# Apply TDIDT on the dataset
attributes = df.columns[:-1]  # Exclude the target column
tree_take_first = tdidt(df, attributes, target)
tree_take_last = tdidt(df, attributes[::-1], target)  # Reverse the order of attributes for "Take Last"

# Extract rules for "Take First" and "Take Last"
rules_take_first = extract_rules(tree_take_first)
rules_take_last = extract_rules(tree_take_last)

# Print the rules
print("\nRules (Take First):")
for rule in rules_take_first:
    print(rule)

print("\nRules (Take Last):")
for rule in rules_take_last:
    print(rule)


ValueError: The truth value of a Index is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [24]:
import pandas as pd

# Define the dataset
data = {
    "Buying": ["med", "med", "med", "med", "med", "low", "low", "low", "low", "low", 
               "low", "low", "low", "low", "low", "low", "low", "low", "low", "low", 
               "low", "low", "low", "low", "low", "low", "low", "low", "low", "low"],
    "Price": ["med", "med", "med", "med", "med", "low", "low", "low", "low", "low", 
              "low", "low", "low", "low", "low", "low", "low", "low", "low", "low", 
              "low", "low", "low", "low", "low", "low", "low", "low", "low", "low"],
    "Maintenance": ["med", "med", "med", "med", "med", "low", "low", "low", "low", "low", 
                    "low", "low", "low", "low", "low", "low", "low", "low", "low", "low", 
                    "low", "low", "low", "low", "low", "low", "low", "low", "low", "low"],
    "Doors": [2, 2, 3, 3, 3, "5more", "5more", "5more", "5more", "5more", "5more", 
              "5more", "5more", "4", "4", "4", "4", "4", "4", "4", "5more", "5more", 
              "5more", "5more", 2, 2, 3, 3, 3, 4],
    "Persons": [4, "more", 4, "more", "more", 2, 2, 4, 4, 4, 4, "more", "more", 4, 4, 
                "more", "more", "4", 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, "more"],
    "Lug_Boot": ["big", "big", "big", "big", "big", "big", "big", "small", "small", "big", 
                 "small", "med", "small", "med", "med", "small", "med", "big", "small", 
                 "big", "small", "small", "small", "small", "med", "small", "med", "small", 
                 "big", "small"],
    "Safety Class": ["vgood", "vgood", "vgood", "vgood", "vgood", "unacc", "unacc", "unacc", 
                     "unacc", "unacc", "unacc", "unacc", "unacc", "good", "good", "good", "good", 
                     "good", "good", "good", "good", "good", "good", "good", "good", "acc", "acc", 
                     "acc", "acc", "acc"]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Save DataFrame to CSV file
df.to_csv('car_evaluation.csv', index=False)

print("CSV file saved successfully.")


CSV file saved successfully.


In [38]:
import pandas as pd

# Specify the path to your existing CSV file
file_path = '/Users/rahatrihan/Desktop/AIUB/data mining/Assignment/car_evaluation.csv'

# Read the existing CSV file
df = pd.read_csv(file_path)

# Display the DataFrame (optional, just to check the contents)
print("Data from CSV file:")
print(df)

# If you want to modify the DataFrame, here is an example:
# Let's say we add a new row to the DataFrame
new_data = {
    "Buying": ["low"],
    "Price": ["med"],
    "Maintenance": ["low"],
    "Doors": [4],
    "Persons": [4],
    "Lug_Boot": ["big"],
    "Safety Class": ["good"]
}
new_row = pd.DataFrame(new_data)

# Append the new row to the existing DataFrame
df = pd.concat([df, new_row], ignore_index=True)

# Write the updated DataFrame back to the same CSV file
df.to_csv(file_path, index=False)

print(f"\nUpdated Data has been written to {file_path}")


Data from CSV file:
   Buying Price Maintenance  Doors Persons Lug_Boot Safety Class
0     med   med         med      2       4      big        vgood
1     med   med         med      2    more      big        vgood
2     med   med         med      3       4      big        vgood
3     med   med         med      3    more      big        vgood
4     med   med         med      3    more      big        vgood
5     low   low         low  5more       2      big        unacc
6     low   low         low  5more       2      big        unacc
7     low   low         low  5more       4    small        unacc
8     low   low         low  5more       4    small        unacc
9     low   low         low  5more       4      big        unacc
10    low   low         low  5more       4    small        unacc
11    low   low         low  5more    more      med        unacc
12    low   low         low  5more    more    small        unacc
13    low   low         low      4       4      med         good
14   

In [42]:
import pandas as pd

# Specify the path to your CSV file
file_path = '/Users/rahatrihan/Desktop/AIUB/data mining/Assignment/car_evaluation.csv'

# Read the CSV file into a DataFrame
data = pd.read_csv(file_path)

# Print the first few rows of the data
print(data.head())


  Buying Price Maintenance Doors Persons Lug_Boot Safety Class
0    med   med         med     2       4      big        vgood
1    med   med         med     2    more      big        vgood
2    med   med         med     3       4      big        vgood
3    med   med         med     3    more      big        vgood
4    med   med         med     3    more      big        vgood


In [1]:
import pandas as pd

# Corrected path to the CSV file
file_path = '/Users/rahatrihan/Desktop/AIUB/data mining/Assignment/car_data.csv'

try:
    # Read the CSV file into a DataFrame
    data = pd.read_csv(file_path)

    # Print the first few rows of the data
    print(data.head())

except FileNotFoundError:
    print(f"The file at {file_path} was not found. Please check the file path.")


The file at /Users/rahatrihan/Desktop/AIUB/data mining/Assignment/car_data.csv was not found. Please check the file path.


In [44]:
    print(data.head())


  Buying Price Maintenance Doors Persons Lug_Boot Safety Class
0    med   med         med     2       4      big        vgood
1    med   med         med     2    more      big        vgood
2    med   med         med     3       4      big        vgood
3    med   med         med     3    more      big        vgood
4    med   med         med     3    more      big        vgood


In [6]:
2+2

4

In [8]:
import pandas as pd

# Corrected path to the CSV file
file_path = '/Users/rahatrihan/Desktop/AIUB/data mining/Assignment/car_evaluation.csv'

try:
    # Read the CSV file into a DataFrame
    data = pd.read_csv(file_path)

    # Print the first few rows of the data
    print(data.head())

except FileNotFoundError:
    print(f"The file at {file_path} was not found. Please check the file path.")


  Buying Price Maintenance Doors Persons Lug_Boot Safety Class
0    med   med         med     2       4      big        vgood
1    med   med         med     2    more      big        vgood
2    med   med         med     3       4      big        vgood
3    med   med         med     3    more      big        vgood
4    med   med         med     3    more      big        vgood


In [10]:
import pandas as pd
import math

# Step 1: Calculate Entropy
def calculate_entropy(data):
    labels = data.iloc[:, -1].value_counts(normalize=True)
    entropy = -sum(labels * labels.apply(lambda x: math.log2(x) if x > 0 else 0))
    return entropy

# Step 2: Calculate Information Gain
def calculate_information_gain(data, feature):
    # Calculate the entropy of the entire dataset
    total_entropy = calculate_entropy(data)
    
    # Group by the feature and calculate weighted entropy of each group
    feature_values = data[feature].value_counts(normalize=True)
    weighted_entropy = sum(feature_values.apply(lambda val: calculate_entropy(data[data[feature] == val])) * feature_values)
    
    # Information Gain = Total Entropy - Weighted Entropy of Feature
    return total_entropy - weighted_entropy

# Step 3: Build the Decision Tree
class TDIDT:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None

    def fit(self, data):
        self.tree = self._build_tree(data)

    def _build_tree(self, data, depth=0):
        # Base Case 1: If all data points belong to the same class
        if len(data.iloc[:, -1].unique()) == 1:
            return data.iloc[0, -1]
        
        # Base Case 2: If there are no features left to split on or if maximum depth is reached
        if len(data.columns) == 1 or (self.max_depth and depth >= self.max_depth):
            return data.iloc[:, -1].mode()[0]
        
        # Step 4: Choose the best feature to split the data based on Information Gain
        best_feature = None
        max_info_gain = -1
        for feature in data.columns[:-1]:  # Exclude the target column
            info_gain = calculate_information_gain(data, feature)
            if info_gain > max_info_gain:
                max_info_gain = info_gain
                best_feature = feature
        
        # Step 5: Create a subtree for each unique value of the best feature
        tree = {best_feature: {}}
        feature_values = data[best_feature].unique()
        for value in feature_values:
            subset = data[data[best_feature] == value].drop(columns=[best_feature])
            tree[best_feature][value] = self._build_tree(subset, depth + 1)
        
        return tree

    def predict(self, row, tree=None):
        if tree is None:
            tree = self.tree

        # If it's a leaf node, return the predicted class
        if isinstance(tree, (str, int, float)):
            return tree
        
        # Traverse the tree recursively
        feature = list(tree.keys())[0]
        feature_value = row[feature]
        if feature_value in tree[feature]:
            return self.predict(row, tree[feature][feature_value])
        else:
            return None  # If the value is not in the tree, return None (you can modify this behavior)

    def predict_all(self, data):
        return data.apply(lambda row: self.predict(row), axis=1)

# Example usage:
if __name__ == "__main__":
    # Step 1: Load your data
    file_path = '/Users/rahatrihan/Desktop/AIUB/data mining/Assignment/car_evaluation.csv'
    data = pd.read_csv(file_path)

    # Step 2: Train TDIDT model
    model = TDIDT(max_depth=3)  # You can adjust the max_depth parameter as needed
    model.fit(data)

    # Step 3: Predict on a new row
    new_row = data.iloc[0]  # You can replace this with any row from your dataset
    prediction = model.predict(new_row)
    print("Prediction for the first row:", prediction)

    # Step 4: Predict on the entire dataset
    predictions = model.predict_all(data)
    print("Predictions on the dataset:", predictions)


Prediction for the first row: vgood
Predictions on the dataset: 0     vgood
1     vgood
2     vgood
3     vgood
4     vgood
5      good
6      good
7      good
8      good
9      good
10     good
11     good
12     good
13     good
14     good
15     good
16     good
17     good
18     good
19     good
20     good
21     good
22     good
23     good
24     good
25     good
26     good
27     good
28     good
29     good
dtype: object


In [12]:
import pandas as pd
import math

# Step 1: Calculate Entropy
def calculate_entropy(data):
    labels = data.iloc[:, -1].value_counts(normalize=True)
    entropy = -sum(labels * labels.apply(lambda x: math.log2(x) if x > 0 else 0))
    return entropy

# Step 2: Calculate Information Gain
def calculate_information_gain(data, feature):
    # Calculate the entropy of the entire dataset
    total_entropy = calculate_entropy(data)
    
    # Group by the feature and calculate weighted entropy of each group
    feature_values = data[feature].value_counts(normalize=True)
    weighted_entropy = sum(feature_values.apply(lambda val: calculate_entropy(data[data[feature] == val])) * feature_values)
    
    # Information Gain = Total Entropy - Weighted Entropy of Feature
    return total_entropy - weighted_entropy

# Step 3: Apply "Take First" and "Take Last" Rules
class TDIDT:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None
        self.rule_count = 0

    def fit(self, data):
        self.tree = self._build_tree(data)

    def _build_tree(self, data, depth=0):
        # Base Case 1: If all data points belong to the same class
        if len(data.iloc[:, -1].unique()) == 1:
            return data.iloc[0, -1]
        
        # Base Case 2: If there are no features left to split on or if maximum depth is reached
        if len(data.columns) == 1 or (self.max_depth and depth >= self.max_depth):
            return data.iloc[:, -1].mode()[0]
        
        # Step 4: Choose the best feature to split the data based on Information Gain
        best_feature = None
        max_info_gain = -1
        for feature in data.columns[:-1]:  # Exclude the target column
            info_gain = calculate_information_gain(data, feature)
            if info_gain > max_info_gain:
                max_info_gain = info_gain
                best_feature = feature
        
        # Step 5: Create a subtree for each unique value of the best feature
        tree = {best_feature: {}}
        feature_values = data[best_feature].unique()
        for value in feature_values:
            subset = data[data[best_feature] == value].drop(columns=[best_feature])
            tree[best_feature][value] = self._build_tree(subset, depth + 1)
            self.rule_count += 1  # Increment rule count
        
        return tree

    def predict(self, row, tree=None):
        if tree is None:
            tree = self.tree

        # If it's a leaf node, return the predicted class
        if isinstance(tree, (str, int, float)):
            return tree
        
        # Traverse the tree recursively
        feature = list(tree.keys())[0]
        feature_value = row[feature]
        if feature_value in tree[feature]:
            return self.predict(row, tree[feature][feature_value])
        else:
            return None  # If the value is not in the tree, return None (you can modify this behavior)

    def predict_all(self, data):
        return data.apply(lambda row: self.predict(row), axis=1)

    def get_rule_count(self):
        return self.rule_count


# Apply "Take First" and "Take Last" rules to the dataset
def apply_take_first_last(data):
    # Take First: The first row of the dataset
    first_row = data.iloc[0]

    # Take Last: The last row of the dataset
    last_row = data.iloc[-1]

    # Print the "Take First" and "Take Last" rules for each attribute
    print("Take First Rule:")
    for column in data.columns[:-1]:  # Exclude the target column
        print(f"{column}: {first_row[column]}")

    print("\nTake Last Rule:")
    for column in data.columns[:-1]:  # Exclude the target column
        print(f"{column}: {last_row[column]}")

# Example usage:
if __name__ == "__main__":
    # Step 1: Load your data
    file_path = '/Users/rahatrihan/Desktop/AIUB/data mining/Assignment/car_evaluation.csv'
    data = pd.read_csv(file_path)

    # Step 2: Apply Take First and Take Last rules
    apply_take_first_last(data)

    # Step 3: Train TDIDT model
    model = TDIDT(max_depth=3)  # You can adjust the max_depth parameter as needed
    model.fit(data)

    # Step 4: Predict on a new row
    new_row = data.iloc[0]  # You can replace this with any row from your dataset
    prediction = model.predict(new_row)
    print("\nPrediction for the first row:", prediction)

    # Step 5: Predict on the entire dataset
    predictions = model.predict_all(data)
    print("\nPredictions on the dataset:", predictions)

    # Step 6: Get the total number of rules in the decision tree
    total_rules = model.get_rule_count()
    print("\nTotal rules in the decision tree:", total_rules)


Take First Rule:
Buying: med
Price: med
Maintenance: med
Doors: 2
Persons: 4
Lug_Boot: big

Take Last Rule:
Buying: low
Price: low
Maintenance: low
Doors: 4
Persons: more
Lug_Boot: small

Prediction for the first row: vgood

Predictions on the dataset: 0     vgood
1     vgood
2     vgood
3     vgood
4     vgood
5      good
6      good
7      good
8      good
9      good
10     good
11     good
12     good
13     good
14     good
15     good
16     good
17     good
18     good
19     good
20     good
21     good
22     good
23     good
24     good
25     good
26     good
27     good
28     good
29     good
dtype: object

Total rules in the decision tree: 4


In [18]:
import pandas as pd
import math
from graphviz import Digraph

# Step 1: Calculate Entropy
def calculate_entropy(data):
    labels = data.iloc[:, -1].value_counts(normalize=True)
    entropy = -sum(labels * labels.apply(lambda x: math.log2(x) if x > 0 else 0))
    return entropy

# Step 2: Calculate Information Gain using Entropy
def calculate_information_gain(data, feature):
    # Calculate the entropy of the entire dataset
    total_entropy = calculate_entropy(data)
    
    # Group by the feature and calculate weighted entropy of each group
    feature_values = data[feature].value_counts(normalize=True)
    weighted_entropy = sum(feature_values.apply(lambda val: calculate_entropy(data[data[feature] == val])) * feature_values)
    
    # Information Gain = Total Entropy - Weighted Entropy of Feature
    return total_entropy - weighted_entropy

# Step 3: Build the Decision Tree and Visualize it using Entropy
class TDIDT:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None
        self.graph = Digraph(comment='Decision Tree')

    def fit(self, data):
        self.tree = self._build_tree(data)
        self._visualize_tree(self.tree)

    def _build_tree(self, data, depth=0):
        # Base Case 1: If all data points belong to the same class
        if len(data.iloc[:, -1].unique()) == 1:
            return data.iloc[0, -1]
        
        # Base Case 2: If there are no features left to split on or if maximum depth is reached
        if len(data.columns) == 1 or (self.max_depth and depth >= self.max_depth):
            return data.iloc[:, -1].mode()[0]
        
        # Step 4: Choose the best feature to split the data based on Information Gain
        best_feature = None
        max_info_gain = -1
        for feature in data.columns[:-1]:  # Exclude the target column
            info_gain = calculate_information_gain(data, feature)
            if info_gain > max_info_gain:
                max_info_gain = info_gain
                best_feature = feature
        
        # Step 5: Create a subtree for each unique value of the best feature
        tree = {best_feature: {}}
        feature_values = data[best_feature].unique()
        for value in feature_values:
            subset = data[data[best_feature] == value].drop(columns=[best_feature])
            tree[best_feature][value] = self._build_tree(subset, depth + 1)
        
        return tree

    def _visualize_tree(self, tree, parent_name=None):
        if isinstance(tree, (str, int, float)):  # It's a leaf node
            self.graph.node(name=str(tree), label=str(tree), shape='ellipse', color='lightblue')
            if parent_name:
                self.graph.edge(parent_name, str(tree))
            return str(tree)
        
        # For non-leaf node, it will be an attribute
        feature = list(tree.keys())[0]
        for feature_value, subtree in tree[feature].items():
            node_name = f"{feature}_{feature_value}"
            self.graph.node(name=node_name, label=f"{feature} = {feature_value}", shape='box', color='lightgreen')
            if parent_name:
                self.graph.edge(parent_name, node_name)
            self._visualize_tree(subtree, node_name)

    def predict(self, row, tree=None):
        if tree is None:
            tree = self.tree

        # If it's a leaf node, return the predicted class
        if isinstance(tree, (str, int, float)):
            return tree
        
        # Traverse the tree recursively
        feature = list(tree.keys())[0]
        feature_value = row[feature]
        if feature_value in tree[feature]:
            return self.predict(row, tree[feature][feature_value])
        else:
            return None  # If the value is not in the tree, return None (you can modify this behavior)

    def predict_all(self, data):
        return data.apply(lambda row: self.predict(row), axis=1)

# Example usage:
if __name__ == "__main__":
    # Load your data
    file_path = '/Users/rahatrihan/Desktop/AIUB/data mining/Assignment/car_evaluation.csv'
    data = pd.read_csv(file_path)

    # Train TDIDT model
    model = TDIDT(max_depth=3)  # You can adjust the max_depth parameter as needed
    model.fit(data)

    # Save and render the decision tree to a file
    model.graph.render('/path/to/save/decision_tree', view=True)  # Change the path as needed

    # Predict on a new row
    new_row = data.iloc[0]  # You can replace this with any row from your dataset
    prediction = model.predict(new_row)
    print("Prediction for the first row:", prediction)

    # Predict on the entire dataset
    predictions = model.predict_all(data)
    print("Predictions on the dataset:", predictions)


OSError: [Errno 30] Read-only file system: '/path'

In [14]:
pip install pandas graphviz


Note: you may need to restart the kernel to use updated packages.


In [24]:
import pandas as pd
import math
from graphviz import Digraph

# Step 1: Calculate Entropy
def calculate_entropy(data):
    labels = data.iloc[:, -1].value_counts(normalize=True)
    entropy = -sum(labels * labels.apply(lambda x: math.log2(x) if x > 0 else 0))
    return entropy

# Step 2: Calculate Information Gain using Entropy
def calculate_information_gain(data, feature):
    # Calculate the entropy of the entire dataset
    total_entropy = calculate_entropy(data)
    
    # Group by the feature and calculate weighted entropy of each group
    feature_values = data[feature].value_counts(normalize=True)
    weighted_entropy = sum(feature_values.apply(lambda val: calculate_entropy(data[data[feature] == val])) * feature_values)
    
    # Information Gain = Total Entropy - Weighted Entropy of Feature
    return total_entropy - weighted_entropy

# Step 3: Build the Decision Tree and Visualize it using Entropy
class TDIDT:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None
        self.rule_count = 0
        self.graph = Digraph(comment='Decision Tree')

    def fit(self, data):
        self.tree = self._build_tree(data)
        self._visualize_tree(self.tree)

    def _build_tree(self, data, depth=0):
        # Base Case 1: If all data points belong to the same class
        if len(data.iloc[:, -1].unique()) == 1:
            return data.iloc[0, -1]
        
        # Base Case 2: If there are no features left to split on or if maximum depth is reached
        if len(data.columns) == 1 or (self.max_depth and depth >= self.max_depth):
            return data.iloc[:, -1].mode()[0]
        
        # Step 4: Choose the best feature to split the data based on Information Gain
        best_feature = None
        max_info_gain = -1
        for feature in data.columns[:-1]:  # Exclude the target column
            info_gain = calculate_information_gain(data, feature)
            if info_gain > max_info_gain:
                max_info_gain = info_gain
                best_feature = feature
        
        # Step 5: Create a subtree for each unique value of the best feature
        tree = {best_feature: {}}
        feature_values = data[best_feature].unique()
        for value in feature_values:
            subset = data[data[best_feature] == value].drop(columns=[best_feature])
            tree[best_feature][value] = self._build_tree(subset, depth + 1)
            self.rule_count += 1  # Increment rule count
        
        return tree

    def _visualize_tree(self, tree, parent_name=None):
        if isinstance(tree, (str, int, float)):  # It's a leaf node
            self.graph.node(name=str(tree), label=str(tree), shape='ellipse', color='lightblue')
            if parent_name:
                self.graph.edge(parent_name, str(tree))
            return str(tree)
        
        # For non-leaf node, it will be an attribute
        feature = list(tree.keys())[0]
        for feature_value, subtree in tree[feature].items():
            node_name = f"{feature}_{feature_value}"
            self.graph.node(name=node_name, label=f"{feature} = {feature_value}", shape='box', color='lightgreen')
            if parent_name:
                self.graph.edge(parent_name, node_name)
            self._visualize_tree(subtree, node_name)

    def predict(self, row, tree=None):
        if tree is None:
            tree = self.tree

        # If it's a leaf node, return the predicted class
        if isinstance(tree, (str, int, float)):
            return tree
        
        # Traverse the tree recursively
        feature = list(tree.keys())[0]
        feature_value = row[feature]
        if feature_value in tree[feature]:
            return self.predict(row, tree[feature][feature_value])
        else:
            return None  # If the value is not in the tree, return None (you can modify this behavior)

    def predict_all(self, data):
        return data.apply(lambda row: self.predict(row), axis=1)

    def get_rule_count(self):
        return self.rule_count


# Step 4: Apply "Take First" and "Take Last" rules to the dataset
def apply_take_first_last(data):
    # Take First: The first row of the dataset
    first_row = data.iloc[0]

    # Take Last: The last row of the dataset
    last_row = data.iloc[-1]

    # Print the "Take First" and "Take Last" rules for each attribute
    print("Take First Rule:")
    for column in data.columns[:-1]:  # Exclude the target column
        print(f"{column}: {first_row[column]}")

    print("\nTake Last Rule:")
    for column in data.columns[:-1]:  # Exclude the target column
        print(f"{column}: {last_row[column]}")

# Example usage:
if __name__ == "__main__":
    # Load your data
    file_path = '/Users/rahatrihan/Desktop/AIUB/data mining/Assignment/car_evaluation.csv'  # Change path if needed
    data = pd.read_csv(file_path)

    # Apply "Take First" and "Take Last" rules
    apply_take_first_last(data)

    # Train TDIDT model
    model = TDIDT(max_depth=None)  # Set max_depth as needed
    model.fit(data)

    # Save and render the decision tree to a file
    model.graph.render('/Users/rahatrihan/Desktop/decision_tree', view=True)  # Change the path if needed

    # Print the total number of rules generated
    print(f"Total number of rules generated: {model.get_rule_count()}")


Take First Rule:
Buying: med
Price: med
Maintenance: med
Doors: 2
Persons: 4
Lug_Boot: big

Take Last Rule:
Buying: low
Price: low
Maintenance: low
Doors: 4
Persons: more
Lug_Boot: small


IsADirectoryError: [Errno 21] Is a directory: '/Users/rahatrihan/Desktop/decision_tree'

In [28]:
import pandas as pd
import math

# Step 1: Calculate Entropy
def calculate_entropy(data):
    labels = data.iloc[:, -1].value_counts(normalize=True)
    entropy = -sum(labels * labels.apply(lambda x: math.log2(x) if x > 0 else 0))
    return entropy

# Step 2: Calculate Information Gain using Entropy
def calculate_information_gain(data, feature):
    total_entropy = calculate_entropy(data)
    
    feature_values = data[feature].value_counts(normalize=True)
    weighted_entropy = sum(feature_values.apply(lambda val: calculate_entropy(data[data[feature] == val])) * feature_values)
    
    return total_entropy - weighted_entropy

# Step 3: TDIDT Algorithm with 'Take First' and 'Take Last'
class TDIDT:
    def __init__(self, approach='take_first', max_depth=None):
        self.max_depth = max_depth
        self.tree = None
        self.rule_count = 0
        self.approach = approach

    def fit(self, data):
        self.tree = self._build_tree(data)
        return self.tree

    def _build_tree(self, data, depth=0):
        if len(data.iloc[:, -1].unique()) == 1:
            return data.iloc[0, -1]
        
        if len(data.columns) == 1 or (self.max_depth and depth >= self.max_depth):
            return data.iloc[:, -1].mode()[0]
        
        best_feature = None
        max_info_gain = -1
        for feature in data.columns[:-1]:  # Exclude target column
            info_gain = calculate_information_gain(data, feature)
            if info_gain > max_info_gain:
                max_info_gain = info_gain
                best_feature = feature
        
        tree = {best_feature: {}}
        feature_values = data[best_feature].unique()

        # Using the 'Take First' or 'Take Last' approach to build the tree
        if self.approach == 'take_first':
            # Select the first row for splitting decision
            first_row = data.iloc[0]
            for value in feature_values:
                subset = data[data[best_feature] == value].drop(columns=[best_feature])
                tree[best_feature][value] = self._build_tree(subset, depth + 1)
                self.rule_count += 1

        elif self.approach == 'take_last':
            # Select the last row for splitting decision
            last_row = data.iloc[-1]
            for value in feature_values:
                subset = data[data[best_feature] == value].drop(columns=[best_feature])
                tree[best_feature][value] = self._build_tree(subset, depth + 1)
                self.rule_count += 1
        
        return tree

# Step 4: Calculate and Display the Number of Rules in a Table
def display_rules_table(take_first_tree, take_last_tree):
    # Create a dictionary to store the results
    rules = {
        "Approach": ["Take First", "Take Last"],
        "Number of Rules": [take_first_tree.rule_count, take_last_tree.rule_count],
        "Generated Decision Tree (Rules)": [str(take_first_tree.tree), str(take_last_tree.tree)]
    }
    
    # Convert dictionary to a DataFrame for better presentation
    df = pd.DataFrame(rules)
    return df

# Main Program
if __name__ == "__main__":
    # Load the dataset (replace with the correct path)
    file_path = '/Users/rahatrihan/Desktop/AIUB/data mining/Assignment/car_evaluation.csv'
    data = pd.read_csv(file_path)

    # Train TDIDT model using 'Take First' and 'Take Last' approaches
    model_take_first = TDIDT(approach='take_first')
    model_take_first.fit(data)
    
    model_take_last = TDIDT(approach='take_last')
    model_take_last.fit(data)

    # Display the number of rules in a table
    result_table = display_rules_table(model_take_first, model_take_last)
    print(result_table)


     Approach  Number of Rules  \
0  Take First               20   
1   Take Last               20   

                     Generated Decision Tree (Rules)  
0  {'Buying': {'med': 'vgood', 'low': {'Price': {...  
1  {'Buying': {'med': 'vgood', 'low': {'Price': {...  


In [30]:
import pandas as pd
import math

# Step 1: Calculate Entropy
def calculate_entropy(data):
    labels = data.iloc[:, -1].value_counts(normalize=True)
    entropy = -sum(labels * labels.apply(lambda x: math.log2(x) if x > 0 else 0))
    return entropy

# Step 2: Calculate Information Gain using Entropy
def calculate_information_gain(data, feature):
    total_entropy = calculate_entropy(data)
    
    feature_values = data[feature].value_counts(normalize=True)
    weighted_entropy = sum(feature_values.apply(lambda val: calculate_entropy(data[data[feature] == val])) * feature_values)
    
    return total_entropy - weighted_entropy

# Step 3: TDIDT Algorithm with 'Take First' and 'Take Last'
class TDIDT:
    def __init__(self, approach='take_first', max_depth=None):
        self.max_depth = max_depth
        self.tree = None
        self.rule_count = 0
        self.approach = approach

    def fit(self, data):
        self.tree = self._build_tree(data)
        return self.tree

    def _build_tree(self, data, depth=0):
        # If all instances have the same label, return the label
        if len(data.iloc[:, -1].unique()) == 1:
            return data.iloc[0, -1]
        
        # If there are no more features to split or max depth is reached, return the majority label
        if len(data.columns) == 1 or (self.max_depth and depth >= self.max_depth):
            return data.iloc[:, -1].mode()[0]
        
        best_feature = None
        max_info_gain = -1
        for feature in data.columns[:-1]:  # Exclude target column
            info_gain = calculate_information_gain(data, feature)
            if info_gain > max_info_gain:
                max_info_gain = info_gain
                best_feature = feature
        
        tree = {best_feature: {}}
        feature_values = data[best_feature].unique()

        # Using the 'Take First' or 'Take Last' approach to build the tree
        if self.approach == 'take_first':
            first_row = data.iloc[0]
            for value in feature_values:
                subset = data[data[best_feature] == value].drop(columns=[best_feature])
                tree[best_feature][value] = self._build_tree(subset, depth + 1)
                self.rule_count += 1

        elif self.approach == 'take_last':
            last_row = data.iloc[-1]
            for value in feature_values:
                subset = data[data[best_feature] == value].drop(columns=[best_feature])
                tree[best_feature][value] = self._build_tree(subset, depth + 1)
                self.rule_count += 1
        
        return tree

    def display_tree(self, tree=None, indent=""):
        """Recursively prints the decision tree."""
        if tree is None:
            tree = self.tree
        
        for feature, branches in tree.items():
            print(f"{indent}{feature}?")
            for value, subtree in branches.items():
                if isinstance(subtree, dict):  # if it's a branch (not a leaf)
                    print(f"{indent}  {value} -> ", end="")
                    self.display_tree(subtree, indent + "    ")
                else:  # if it's a leaf (final classification)
                    print(f"{indent}  {value} -> {subtree}")

# Step 4: Calculate and Display the Number of Rules in a Table
def display_rules_table(take_first_tree, take_last_tree):
    rules = {
        "Approach": ["Take First", "Take Last"],
        "Number of Rules": [take_first_tree.rule_count, take_last_tree.rule_count],
        "Generated Decision Tree (Rules)": [str(take_first_tree.tree), str(take_last_tree.tree)]
    }
    
    df = pd.DataFrame(rules)
    return df

# Main Program
if __name__ == "__main__":
    # Load the dataset (replace with the correct path)
    file_path = '/Users/rahatrihan/Desktop/AIUB/data mining/Assignment/car_evaluation.csv'
    data = pd.read_csv(file_path)

    # Train TDIDT model using 'Take First' and 'Take Last' approaches
    model_take_first = TDIDT(approach='take_first')
    model_take_first.fit(data)
    
    model_take_last = TDIDT(approach='take_last')
    model_take_last.fit(data)

    # Display the generated trees
    print("Decision Tree using 'Take First' approach:")
    model_take_first.display_tree()

    print("\nDecision Tree using 'Take Last' approach:")
    model_take_last.display_tree()

    # Display the number of rules in a table
    result_table = display_rules_table(model_take_first, model_take_last)
    print("\nNumber of Rules Generated:")
    print(result_table)


Decision Tree using 'Take First' approach:
Buying?
  med -> vgood
  low ->     Price?
      low ->         Maintenance?
          low ->             Doors?
              5more ->                 Persons?
                  2 -> unacc
                  4 ->                     Lug_Boot?
                      small -> good
                      big -> unacc
                  more -> unacc
              4 ->                 Persons?
                  4 -> good
                  more ->                     Lug_Boot?
                      small -> acc
                      med -> good
              2 ->                 Persons?
                  4 ->                     Lug_Boot?
                      med -> good
                      small -> acc
              3 -> acc

Decision Tree using 'Take Last' approach:
Buying?
  med -> vgood
  low ->     Price?
      low ->         Maintenance?
          low ->             Doors?
              5more ->                 Persons?
                  2 -

In [34]:
import pandas as pd
import math
from graphviz import Digraph

# Step 1: Calculate Entropy
def calculate_entropy(data):
    labels = data.iloc[:, -1].value_counts(normalize=True)
    entropy = -sum(labels * labels.apply(lambda x: math.log2(x) if x > 0 else 0))
    return entropy

# Step 2: Calculate Information Gain using Entropy
def calculate_information_gain(data, feature):
    total_entropy = calculate_entropy(data)
    
    feature_values = data[feature].value_counts(normalize=True)
    weighted_entropy = sum(feature_values.apply(lambda val: calculate_entropy(data[data[feature] == val])) * feature_values)
    
    return total_entropy - weighted_entropy

# Step 3: TDIDT Algorithm with 'Take First' and 'Take Last'
class TDIDT:
    def __init__(self, approach='take_first', max_depth=None):
        self.max_depth = max_depth
        self.tree = None
        self.rule_count = 0
        self.approach = approach
        self.dot = Digraph()

    def fit(self, data):
        self.tree = self._build_tree(data)
        return self.tree

    def _build_tree(self, data, depth=0):
        # If all instances have the same label, return the label
        if len(data.iloc[:, -1].unique()) == 1:
            return data.iloc[0, -1]
        
        # If there are no more features to split or max depth is reached, return the majority label
        if len(data.columns) == 1 or (self.max_depth and depth >= self.max_depth):
            return data.iloc[:, -1].mode()[0]
        
        best_feature = None
        max_info_gain = -1
        for feature in data.columns[:-1]:  # Exclude target column
            info_gain = calculate_information_gain(data, feature)
            if info_gain > max_info_gain:
                max_info_gain = info_gain
                best_feature = feature
        
        tree = {best_feature: {}}
        feature_values = data[best_feature].unique()

        # Using the 'Take First' or 'Take Last' approach to build the tree
        if self.approach == 'take_first':
            for value in feature_values:
                subset = data[data[best_feature] == value].drop(columns=[best_feature])
                tree[best_feature][value] = self._build_tree(subset, depth + 1)
                self.rule_count += 1

        elif self.approach == 'take_last':
            for value in feature_values:
                subset = data[data[best_feature] == value].drop(columns=[best_feature])
                tree[best_feature][value] = self._build_tree(subset, depth + 1)
                self.rule_count += 1
        
        return tree

    def display_tree(self, tree=None, parent=None, graph=None):
        """Recursively adds nodes and edges to the graphviz Digraph."""
        if tree is None:
            tree = self.tree

        if graph is None:
            graph = self.dot

        # If it's a dictionary, it means it's a non-leaf node
        if isinstance(tree, dict):
            for feature, branches in tree.items():
                # For each feature, create a node
                node_name = feature
                graph.node(node_name, label=feature)
                if parent:
                    graph.edge(parent, node_name)

                for value, subtree in branches.items():
                    child_node_name = f"{feature}_{value}"
                    graph.node(child_node_name, label=f"{value}")
                    graph.edge(node_name, child_node_name)
                    self.display_tree(subtree, child_node_name, graph)
        else:
            # If it's a leaf node (final class label)
            graph.node(f"{parent}_leaf", label=f"{tree}")
            graph.edge(parent, f"{parent}_leaf")
        
        return graph

    def render_tree(self, filename='decision_tree'):
        """Render the decision tree to a file."""
        self.dot = self.display_tree(self.tree, graph=self.dot)
        self.dot.render(filename, format='png', cleanup=True)

# Main Program
if __name__ == "__main__":
    # Load the dataset (replace with the correct path)
    file_path = '/Users/rahatrihan/Desktop/AIUB/data mining/Assignment/car_evaluation.csv'
    data = pd.read_csv(file_path)

    # Train TDIDT model using 'Take First' approach
    model_take_first = TDIDT(approach='take_first')
    model_take_first.fit(data)
    
    # Render and save the decision tree as a PNG file
    model_take_first.render_tree('decision_tree_take_first')  # Save as PNG

    # Train TDIDT model using 'Take Last' approach
    model_take_last = TDIDT(approach='take_last')
    model_take_last.fit(data)
    
    # Render and save the decision tree as a PNG file
    model_take_last.render_tree('decision_tree_take_last')  # Save as PNG

    print("Decision trees have been generated and saved as PNG files.")


ExecutableNotFound: failed to execute PosixPath('dot'), make sure the Graphviz executables are on your systems' PATH

In [36]:
/bin/bash -c "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)"


NameError: name 'bash' is not defined

In [52]:
import pandas as pd
import math
from graphviz import Digraph

# Calculate Entropy
def calculate_entropy(data):
    labels = data.iloc[:, -1].value_counts(normalize=True)
    entropy = -sum(labels * labels.apply(lambda x: math.log2(x) if x > 0 else 0))
    return entropy

# Calculate Information Gain using Entropy
def calculate_information_gain(data, feature):
    total_entropy = calculate_entropy(data)
    
    feature_values = data[feature].value_counts(normalize=True)
    weighted_entropy = sum(feature_values.apply(lambda val: calculate_entropy(data[data[feature] == val])) * feature_values)
    
    return total_entropy - weighted_entropy

# TDIDT Algorithm with 'Take First' and 'Take Last'
class TDIDT:
    def __init__(self, approach='take_first', max_depth=None):
        self.max_depth = max_depth
        self.tree = None
        self.rule_count = 0
        self.approach = approach
        self.dot = Digraph()

    def fit(self, data):
        self.tree = self._build_tree(data)
        return self.tree

    def _build_tree(self, data, depth=0):
        # If all instances have the same label, return the label
        if len(data.iloc[:, -1].unique()) == 1:
            return data.iloc[0, -1]
        
        # If there are no more features to split or max depth is reached, return the majority label
        if len(data.columns) == 1 or (self.max_depth and depth >= self.max_depth):
            return data.iloc[:, -1].mode()[0]
        
        best_feature = None
        max_info_gain = -1
        for feature in data.columns[:-1]:  # Exclude target column
            info_gain = calculate_information_gain(data, feature)
            if info_gain > max_info_gain:
                max_info_gain = info_gain
                best_feature = feature
        
        tree = {best_feature: {}}
        feature_values = data[best_feature].unique()

        # Using the 'Take First' or 'Take Last' approach to build the tree
        if self.approach == 'take_first':
            for value in feature_values:
                subset = data[data[best_feature] == value].drop(columns=[best_feature])
                tree[best_feature][value] = self._build_tree(subset, depth + 1)
                self.rule_count += 1

        elif self.approach == 'take_last':
            for value in feature_values:
                subset = data[data[best_feature] == value].drop(columns=[best_feature])
                tree[best_feature][value] = self._build_tree(subset, depth + 1)
                self.rule_count += 1
        
        return tree

    def display_tree(self, tree=None, parent=None, graph=None):
        """Recursively adds nodes and edges to the graphviz Digraph."""
        if tree is None:
            tree = self.tree

        if graph is None:
            graph = self.dot

        # If it's a dictionary, it means it's a non-leaf node
        if isinstance(tree, dict):
            for feature, branches in tree.items():
                # For each feature, create a node
                node_name = feature
                graph.node(node_name, label=feature)
                if parent:
                    graph.edge(parent, node_name)

                for value, subtree in branches.items():
                    child_node_name = f"{feature}_{value}"
                    graph.node(child_node_name, label=f"{value}")
                    graph.edge(node_name, child_node_name)
                    self.display_tree(subtree, child_node_name, graph)
        else:
            # If it's a leaf node (final class label)
            graph.node(f"{parent}_leaf", label=f"{tree}")
            graph.edge(parent, f"{parent}_leaf")
        
        return graph

    def render_tree(self, filename='decision_tree'):
        """Render the decision tree to a file."""
        self.dot = self.display_tree(self.tree, graph=self.dot)
        self.dot.render(filename, format='png', cleanup=True)

# Main Program
if __name__ == "__main__":
    # Load the dataset (replace with the correct path)
    file_path = '/Users/rahatrihan/Desktop/AIUB/data mining/Assignment/car_evaluation.csv'
    data = pd.read_csv(file_path)

    # Train TDIDT model using 'Take First' approach
    model_take_first = TDIDT(approach='take_first')
    model_take_first.fit(data)
    print(f"Number of rules generated (Take First): {model_take_first.rule_count}")
    model_take_first.render_tree('decision_tree_take_first')

    # Train TDIDT model using 'Take Last' approach
    model_take_last = TDIDT(approach='take_last')
    model_take_last.fit(data)
    print(f"Number of rules generated (Take Last): {model_take_last.rule_count}")
    model_take_last.render_tree('decision_tree_take_last')


Number of rules generated (Take First): 21


ExecutableNotFound: failed to execute PosixPath('dot'), make sure the Graphviz executables are on your systems' PATH

In [50]:
pip install graphviz


Note: you may need to restart the kernel to use updated packages.


In [56]:
import pandas as pd
import math

# Step 1: Calculate Entropy
def calculate_entropy(data):
    labels = data.iloc[:, -1].value_counts(normalize=True)
    entropy = -sum(labels * labels.apply(lambda x: math.log2(x) if x > 0 else 0))
    return entropy

# Step 2: Calculate Information Gain using Entropy
def calculate_information_gain(data, feature):
    total_entropy = calculate_entropy(data)
    
    feature_values = data[feature].value_counts(normalize=True)
    weighted_entropy = sum(feature_values.apply(lambda val: calculate_entropy(data[data[feature] == val])) * feature_values)
    
    return total_entropy - weighted_entropy

# Step 3: TDIDT Algorithm with 'Take First' and 'Take Last'
class TDIDT:
    def __init__(self, approach='take_first', max_depth=None):
        self.max_depth = max_depth
        self.tree = None
        self.rule_count = 0
        self.approach = approach

    def fit(self, data):
        self.tree = self._build_tree(data)
        return self.tree

    def _build_tree(self, data, depth=0):
        # If all instances have the same label, return the label
        if len(data.iloc[:, -1].unique()) == 1:
            return data.iloc[0, -1]
        
        # If there are no more features to split or max depth is reached, return the majority label
        if len(data.columns) == 1 or (self.max_depth and depth >= self.max_depth):
            return data.iloc[:, -1].mode()[0]
        
        best_feature = None
        max_info_gain = -1
        for feature in data.columns[:-1]:  # Exclude target column
            info_gain = calculate_information_gain(data, feature)
            if info_gain > max_info_gain:
                max_info_gain = info_gain
                best_feature = feature
        
        tree = {best_feature: {}}
        feature_values = data[best_feature].unique()

        # Using the 'Take First' or 'Take Last' approach to build the tree
        if self.approach == 'take_first':
            for value in feature_values:
                subset = data[data[best_feature] == value].drop(columns=[best_feature])
                tree[best_feature][value] = self._build_tree(subset, depth + 1)
                self.rule_count += 1

        elif self.approach == 'take_last':
            for value in feature_values:
                subset = data[data[best_feature] == value].drop(columns=[best_feature])
                tree[best_feature][value] = self._build_tree(subset, depth + 1)
                self.rule_count += 1
        
        return tree

    def count_rules(self, tree=None):
        """Recursively count the number of leaf nodes (rules) in the tree."""
        if tree is None:
            tree = self.tree
        
        rule_count = 0
        
        if isinstance(tree, dict):  # Non-leaf node
            for feature, branches in tree.items():
                for value, subtree in branches.items():
                    rule_count += self.count_rules(subtree)  # Recursive call
        else:
            rule_count += 1  # Leaf node, one rule
        
        return rule_count

# Main Program
if __name__ == "__main__":
    # Load the dataset (replace with the correct path)
    file_path = '/Users/rahatrihan/Desktop/AIUB/data mining/Assignment/car_evaluation.csv'
    data = pd.read_csv(file_path)

    # Train TDIDT model using 'Take First' approach
    model_take_first = TDIDT(approach='take_first')
    model_take_first.fit(data)
    rules_take_first = model_take_first.count_rules()

    # Train TDIDT model using 'Take Last' approach
    model_take_last = TDIDT(approach='take_last')
    model_take_last.fit(data)
    rules_take_last = model_take_last.count_rules()

    print(f"Number of rules generated (Take First approach): {rules_take_first}")
    print(f"Number of rules generated (Take Last approach): {rules_take_last}")


Number of rules generated (Take First approach): 12
Number of rules generated (Take Last approach): 12


In [58]:
import pandas as pd
import math

# Step 1: Calculate Entropy for a dataset
def calculate_entropy(data):
    # Calculate the proportion of each class
    labels = data.iloc[:, -1].value_counts(normalize=True)
    # Calculate entropy: H(D) = - sum(p(x) * log2(p(x))) for all x in labels
    entropy = -sum(labels * labels.apply(lambda x: math.log2(x) if x > 0 else 0))
    return entropy

# Step 2: Calculate Information Gain for a feature
def calculate_information_gain(data, feature):
    total_entropy = calculate_entropy(data)
    feature_values = data[feature].value_counts(normalize=True)
    weighted_entropy = sum(feature_values.apply(lambda val: calculate_entropy(data[data[feature] == val])) * feature_values)
    
    return total_entropy - weighted_entropy

# Step 3: TDIDT Algorithm to build a Decision Tree
class TDIDT:
    def __init__(self, approach='take_first'):
        self.tree = None
        self.approach = approach

    def fit(self, data):
        self.tree = self._build_tree(data)
        return self.tree

    def _build_tree(self, data):
        # If all instances have the same label, return the label
        if len(data.iloc[:, -1].unique()) == 1:
            return data.iloc[0, -1]
        
        # If there are no more features to split on, return the majority label
        if len(data.columns) == 1:
            return data.iloc[:, -1].mode()[0]
        
        best_feature = None
        max_info_gain = -1
        
        # Find the best feature to split on
        for feature in data.columns[:-1]:  # Exclude target column
            info_gain = calculate_information_gain(data, feature)
            if info_gain > max_info_gain:
                max_info_gain = info_gain
                best_feature = feature
        
        tree = {best_feature: {}}
        feature_values = data[best_feature].unique()

        # Recursively build the tree based on the selected feature
        for value in feature_values:
            subset = data[data[best_feature] == value].drop(columns=[best_feature])
            tree[best_feature][value] = self._build_tree(subset)
        
        return tree

    def print_tree(self, tree=None, indent=""):
        """Print the decision tree in a human-readable format."""
        if tree is None:
            tree = self.tree
        
        if isinstance(tree, dict):
            for feature, branches in tree.items():
                for value, subtree in branches.items():
                    print(f"{indent}{feature} = {value}?")
                    self.print_tree(subtree, indent + "  ")
        else:
            print(f"{indent}Predict: {tree}")

# Load the dataset (make sure to replace with your dataset path)
file_path = '/Users/rahatrihan/Desktop/AIUB/data mining/Assignment/car_evaluation.csv'
data = pd.read_csv(file_path)

# Train a TDIDT model using 'Take First' approach
model = TDIDT(approach='take_first')
model.fit(data)

# Print the generated decision tree
model.print_tree()



Buying = med?
  Predict: vgood
Buying = low?
  Price = low?
    Maintenance = low?
      Doors = 5more?
        Persons = 2?
          Predict: unacc
        Persons = 4?
          Lug_Boot = small?
            Predict: good
          Lug_Boot = big?
            Predict: unacc
        Persons = more?
          Predict: unacc
      Doors = 4?
        Persons = 4?
          Predict: good
        Persons = more?
          Lug_Boot = small?
            Predict: acc
          Lug_Boot = med?
            Predict: good
      Doors = 2?
        Persons = 4?
          Lug_Boot = med?
            Predict: good
          Lug_Boot = small?
            Predict: acc
      Doors = 3?
        Predict: acc
  Price = med?
    Predict: good
